Padrón: 110197 -> último nro = 7 -> me corresponde la categoría deportes

Tablas dadas listando sus columnas:
# Archivo: productos_con_demanda.csv 
- producto_id: ID único del producto, compuesto por el prefijo de la familia y un número (ej. ELE1, 
MUE35). 
- familia: Nombre de la categoría del producto (Electrónicos, Muebles, etc.). 
- volumen_m3: Volumen por unidad del producto, en metros cúbicos. 
- costo_M1_usd: Costo por unidad en USD al mayorista M1 (más cercano, más caro). 
- costo_M2_usd: Costo por unidad en USD al mayorista M2 (referencia intermedia). 
- costo_M3_usd: Costo por unidad en USD al mayorista M3 (más lejano, más barato). 
- pdv_1 a pdv_5: Cantidad demandada del producto en cada punto de venta (en unidades). 
- demanda_total: Total de unidades demandadas sumando los 5 puntos de venta. 
- volumen_total: Volumen total asociado a la demanda (volumen_m3 × demanda_total). 
# Archivo: depositos_con_costos.csv 
- familia: Familia de productos a la que pertenece el depósito. 
- deposito: Identificador del depósito (por ejemplo D1_ELE, D2_MUE). 
- capacidad_m3: Capacidad total del depósito en metros cúbicos. 
- pdv_1 a pdv_5: Costo de transporte por metro cúbico desde ese depósito a cada punto de venta 
(USD/10m³). Los costos son más bajos hacia centros cercanos. 
# Archivo: costos_mayoristas.csv 
- mayorista: Identificador del mayorista (M1, M2, M3). 
- D1_XXX, D2_XXX: Costo de transporte por metro cúbico desde el mayorista al depósito para la familia de bienes especificada.(USD/m³). En mi caso sería D1_DEP y D2_DEP.
- pdv_1 a pdv_5: Costo de transporte por metro cúbico desde el mayorista directamente al punto 
de venta. Estos costos son significativamente más altos que los indirectos (vía depósito). 

# Datos:

Se presenta una situación logística en la que una empresa debe abastecer la demanda de 
distintos productos en cinco centros de venta, minimizando los costos totales. Los costos 
considerados incluyen tanto el costo de adquisición de los productos como los costos de 
transporte desde los mayoristas hasta los centros de venta (ya sea de forma directa o a través de 
depósitos intermedios). 

- Existen 10 familias de productos, cada una compuesta por 100 productos diferentes (total: 
1000 productos). 
- Hay 3 mayoristas (centros de compra) que ofrecen todos los productos con distintos costos. 
- Para cada familia, se cuenta con 2 depósitos dedicados exclusivamente a esa familia. Cada 
depósito tiene una capacidad máxima de volumen (en m³). 
- La demanda está definida para cada producto en cada uno de los 5 centros de venta. 
- Se permite realizar un solo ciclo de compra (una única asignación por producto en este 
modelo). 
- El objetivo es satisfacer la demanda de cada centro de venta al menor costo posible, teniendo 
en cuenta: 
    -  El costo de compra a cada mayorista por producto. 
    -  El costo de transporte desde el mayorista a depósito. 
    -  El costo de transporte desde el depósito al centro de venta. 
    -  El costo directo de transporte desde el mayorista al centro de venta (en caso de no utilizar 
depósitos).

# Consignas

Los archivos a entregar debe representar la orden de compra a los 3 mayoristas para los productos de
la familia asignada. Los productos se compran a los 3 mayoristas, por lo tanto debería existir una
orden de compra a M1, M2 y M3; cada uno en un archivo .csv distinto.


Cada fila representa un producto y debe contener las siguientes columnas:
- producto_id: ID del producto.
- cantidad_total: cantidad total a comprar del producto al mayorista correspondiente.
- D1_xxx: cantidad a enviar al depósito 1 (ej. D1_ELE).
- D2_xxx: cantidad a enviar al depósito 2.
- pdv_1 a pdv_5: cantidad a enviar directamente desde el mayorista correspondiente al punto de venta
correspondiente (sin pasar por depósitos).


Ej header csv
producto_id,cantidad_total,D1_ELE, D2_ELE,pdv_1,pdv_2,pdv_3,pdv_4,pdv_5
ELE01,12,5,4,1,1,1,0,0
ELE02,7,0,3,2,1,1,0,0
ELE03,0,0,0,0,0,0,0,0
ELE04,20,10,8,1,1,0,0,0

1. El costo total de la familia, en USD 
2. Cuanto esta dispuesto a pagar por 1m3 mas de volumen en el deposito 1. (Lo mismo para 
1000, y 100000m3) en USD 
3. Subir el código o material suplementario usado, python, excel, colab, etc. 

# Código:

In [1]:
# Importo Pandas, Numpy, y lo que creo que voy a necesitar de PuLP (capaz algo más después)
import pandas as pd
import numpy as np

In [2]:
# Armo mis dataframes
productos_df = pd.read_csv('data/productos_con_demanda.csv')
depositos_df = pd.read_csv('data/depositos_con_costos.csv')
mayoristas_df = pd.read_csv('data/costos_mayoristas.csv')

# Reviso que se hayan importado bien
print("Productos DataFrame:")
print(productos_df.head())
print("\nDepositos DataFrame:")
print(depositos_df.head())
print("\nMayoristas DataFrame:")
print(mayoristas_df.head())

Productos DataFrame:
  producto_id       familia  volumen_m3  costo_M1_usd  costo_M2_usd  \
0        ELE1  Electrónicos       0.049        346.39        314.90   
1        ELE2  Electrónicos       0.060        339.21        308.37   
2        ELE3  Electrónicos       0.045        314.71        286.10   
3        ELE4  Electrónicos       0.043        315.88        287.16   
4        ELE5  Electrónicos       0.051        312.04        283.67   

   costo_M3_usd  pdv_1  pdv_2  pdv_3  pdv_4  pdv_5  demanda_total  \
0        267.66     62     43      0      0      0            105   
1        262.11     59      0     96     77      0            232   
2        243.18      1     33     19      0     45             98   
3        244.09     10     23     70      0     31            134   
4        241.12     44      0      0     93     96            233   

   volumen_total  
0          5.145  
1         13.920  
2          4.410  
3          5.762  
4         11.883  

Depositos DataFrame:
 

In [3]:
# Armo el filtro de la familia de productos que me corresponde (En este caso "Deportes" por mi padrón)
assigned_family = "Deportes"

# Filtro en productos y en depositos
filtered_productos_df = productos_df[productos_df['familia'] == assigned_family]
filtered_depositos_df = depositos_df[depositos_df['familia'] == assigned_family]
filtered_mayoristas_df = mayoristas_df[["mayorista", "D1_DEP", "D2_DEP", "pdv_1", "pdv_2", "pdv_3", "pdv_4", "pdv_5"]]


Los imprimo para ver si quedaron bien:

In [4]:
filtered_productos_df

,producto_id,familia,volumen_m3,costo_M1_usd,costo_M2_usd,costo_M3_usd,pdv_1,pdv_2,pdv_3,pdv_4,pdv_5,demanda_total,volumen_total
700,DEP1,Deportes,0.232,105.24,95.67,81.32,0,0,0,69,25,94,21.808
701,DEP2,Deportes,0.180,103.43,94.03,79.93,5,92,7,99,27,230,41.400
702,DEP3,Deportes,0.186,129.94,118.13,100.41,98,0,0,61,0,159,29.574
703,DEP4,Deportes,0.156,108.92,99.02,84.17,44,19,0,70,0,133,20.748
704,DEP5,Deportes,0.211,114.36,103.96,88.37,19,34,26,13,84,176,37.136
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,DEP96,Deportes,0.200,101.31,92.10,78.28,55,30,67,27,0,179,35.800
796,DEP97,Deportes,0.219,98.49,89.54,76.11,82,45,0,95,32,254,55.626
797,DEP98,Deportes,0.174,112.12,101.93,86.64,0,72,74,97,29,272,47.328
798,DEP99,Deportes,0.189,127.64,116.04,98.63,0,36,36,0,17,89,16.821


In [5]:
filtered_depositos_df

,familia,deposito,capacidad_m3,pdv_1,pdv_2,pdv_3,pdv_4,pdv_5
4,Deportes,D1_DEP,1500,11.32,11.78,17.97,17.78,15.13
5,Deportes,D2_DEP,1300,18.22,16.45,16.79,9.80,10.03


In [6]:
filtered_mayoristas_df

,mayorista,D1_DEP,D2_DEP,pdv_1,pdv_2,pdv_3,pdv_4,pdv_5
0,M1,14.88,24.75,29.79,33.88,26.08,27.21,26.10
1,M2,20.04,17.15,32.92,32.76,33.67,36.24,39.47
2,M3,19.22,17.24,30.16,29.16,39.95,36.26,34.67


In [7]:
# Saco la columna familia que se volvió redundante
filtered_depositos_df = filtered_depositos_df.drop(columns=["familia"])

# Hago lo mismo para productos
filtered_productos_df = filtered_productos_df.drop(columns=["familia"])

Paso estos 3 filtrados (con sólo la data que me es relevante a nuevos .csv para poder reimportarlos y retomar de ahí si cometo un error):

In [8]:
filtered_productos_df.to_csv('Archivos Intermedios/filtered_productos.csv', index=False)
filtered_depositos_df.to_csv('Archivos Intermedios/filtered_depositos.csv', index=False)
filtered_mayoristas_df.to_csv('Archivos Intermedios/filtered_mayoristas.csv', index=False)

Entonces, ahora que tengo mis archivos filtrados con sólo los datos relevantes; puedo empezar a plantear el problema

# **Paso a ETL 2 para separar las transformaciones de "preparación de los archivos" con las de "resolución del problema"**